<a href="https://colab.research.google.com/github/teamgaon/KLUE/blob/main/220223_ji_trans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!pip install pororo

     |████████████████████████████████| 3.5 MB 8.4 MB/s 
     |████████████████████████████████| 6.8 MB 36.7 MB/s 
     |████████████████████████████████| 895 kB 63.7 MB/s 
     |████████████████████████████████| 596 kB 67.9 MB/s 
     |████████████████████████████████| 67 kB 7.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 958 kB 8.1 MB/s 
     |████████████████████████████████| 138 kB 72.8 MB/s 
     |████████████████████████████████| 359 kB 51.1 MB/s 
     |████████████████████████████████| 3.6 MB 53.0 MB/s 
     |████████████████████████████████| 55 kB 5.4 MB/s 
     |████████████████████████████████| 58 kB 7.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into 

In [3]:
import sys
import numpy as np
import pandas as pd
import random
from tqdm.notebook import tqdm
import os
import re
import time
from collections import Counter

import transformers
from transformers import BertTokenizer,AdamWeightDecay,TFRobertaModel,TFBertModel

import tensorflow as tf
import keras
from keras.callbacks import EarlyStopping,ModelCheckpoint

import sklearn
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.model_selection import StratifiedKFold

import pororo
from pororo import Pororo

import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [4]:
print(tf.__version__)
print(np.__version__)
print(pd.__version__)
print(transformers.__version__)
print(keras.__version__)
print(sklearn.__version__)
print(pororo.__version__)
print(selenium.__version__)

2.8.0
1.21.5
1.3.5
4.16.2
2.8.0
1.0.2
0.4.2
4.1.0


In [5]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [6]:
kakao_snli = pd.read_csv('/content/drive/MyDrive/KLUE/snli_1.0_train.ko.tsv', sep='\t', encoding='utf-8')
kakao_snli = kakao_snli[:100000]

PATH =  '/content/drive/MyDrive/KLUE'
test = pd.read_csv(os.path.join(PATH, 'test_data.csv'), encoding='utf-8')

kakao_dev = pd.read_csv('/content/drive/MyDrive/KLUE/xnli.dev.ko.tsv', sep='\t', encoding='utf-8')
kakao_test = pd.read_csv('/content/drive/MyDrive/KLUE/xnli.test.ko.tsv', sep='\t', encoding='utf-8')
klue_dev = pd.read_json('/content/drive/MyDrive/KLUE/klue-nli-v1.1_dev.json')
klue_train = pd.read_json('/content/drive/MyDrive/KLUE/klue-nli-v1.1_train.json')
temp = pd.read_csv(os.path.join(PATH, 'train_data.csv'), encoding='utf-8')

klue_dev = klue_dev[['premise', 'hypothesis', 'gold_label']]
klue_train = klue_train[['premise', 'hypothesis', 'gold_label']]
klue_dev.rename(columns = {'gold_label':'label'}, inplace=True)
klue_train.rename(columns = {'gold_label':'label'}, inplace=True)

train = pd.concat([kakao_dev,kakao_test, kakao_snli])
train.rename(columns = {'sentence1':'premise','sentence2':'hypothesis','gold_label':'label'},inplace=True)
train = pd.concat([temp, train, klue_dev, klue_train], axis=0)
train = train.reset_index(drop=True)
train['index'] = train.index
train = train.dropna()
train

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,index,premise,hypothesis,label
0,0,"씨름은 상고시대로부터 전해져 내려오는 남자들의 대표적인 놀이로서, 소년이나 장정들이...",씨름의 여자들의 놀이이다.,contradiction
1,1,"삼성은 자작극을 벌인 2명에게 형사 고소 등의 법적 대응을 검토 중이라고 하였으나,...",자작극을 벌인 이는 3명이다.,contradiction
2,2,이를 위해 예측적 범죄예방 시스템을 구축하고 고도화한다.,예측적 범죄예방 시스템 구축하고 고도화하는 것은 목적이 있기 때문이다.,entailment
3,3,광주광역시가 재개발 정비사업 원주민들에 대한 종합대책을 마련하는 등 원주민 보호에 ...,원주민들은 종합대책에 만족했다.,neutral
4,4,"진정 소비자와 직원들에게 사랑 받는 기업으로 오래 지속되고 싶으면, 이런 상황에서는...",이런 상황에서 책임 있는 모습을 보여주는 기업은 아주 드물다.,neutral
...,...,...,...,...
159516,159516,"힐링푸드 전문가 양성 교육 참여, 개발 음식 활용 등의 혜택도 주어진다.",개발 음식 활용 혜택만 주어진다.,contradiction
159517,159517,"힐링푸드 전문가 양성 교육 참여, 개발 음식 활용 등의 혜택도 주어진다.",힐링푸드 전문가에게 혜택이 주어진다.,neutral
159518,159518,"힐링푸드 전문가 양성 교육 참여, 개발 음식 활용 등의 혜택도 주어진다.",혜택에는 힐링푸드 전문가 양성 교육 참여가 있다.,entailment
159519,159519,힛걸 진심 최고다 그 어떤 히어로보다 멋지다,힛걸 그 어떤 히어로보다 별로다.,contradiction


In [7]:
# Setting results path
data_path=trans_path = "/content/drive/MyDrive/KLUE/trans/"

In [15]:
def chrome_setting():
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Chrome('chromedriver', options=chrome_options)
  return driver

In [16]:
driver=chrome_setting()

In [17]:
# Crawling
def kor_to_trans(text_data, trans_lang,start_index,final_index):

  target_present = EC.presence_of_element_located((By.XPATH, '//*[@id="txtTarget"]'))

  for i in tqdm(range(start_index,final_index)): 
    
    # if (i!=0)&(i%99==0):
    #   time.sleep(2)
    #   print('{}th : '.format(i), backtrans)
    #   np.save(data_path+'kor_to_eng_train_{}_{}.npy'.format(start_index,final_index),trans_list)
    
    try:
      driver.get('https://papago.naver.com/?sk=ko&tk='+trans_lang+'&st='+text_data[i])
      time.sleep(1.5)
      element=WebDriverWait(driver, 10).until(target_present)
      time.sleep(0.1)
      backtrans = element.text 

      if (backtrans=='')|(backtrans==' '):
        element=WebDriverWait(driver, 10).until(target_present)
        backtrans = element.text 
        trans_list.append(backtrans)
      else:
        trans_list.append(backtrans)
    
    except:
      trans_list.append('')


In [20]:
feature = ['premise','hypothesis']

In [23]:
train

,index,premise,hypothesis,label
0,0,"씨름은 상고시대로부터 전해져 내려오는 남자들의 대표적인 놀이로서, 소년이나 장정들이...",씨름의 여자들의 놀이이다.,contradiction
1,1,"삼성은 자작극을 벌인 2명에게 형사 고소 등의 법적 대응을 검토 중이라고 하였으나,...",자작극을 벌인 이는 3명이다.,contradiction
2,2,이를 위해 예측적 범죄예방 시스템을 구축하고 고도화한다.,예측적 범죄예방 시스템 구축하고 고도화하는 것은 목적이 있기 때문이다.,entailment
3,3,광주광역시가 재개발 정비사업 원주민들에 대한 종합대책을 마련하는 등 원주민 보호에 ...,원주민들은 종합대책에 만족했다.,neutral
4,4,"진정 소비자와 직원들에게 사랑 받는 기업으로 오래 지속되고 싶으면, 이런 상황에서는...",이런 상황에서 책임 있는 모습을 보여주는 기업은 아주 드물다.,neutral
...,...,...,...,...
159516,159516,"힐링푸드 전문가 양성 교육 참여, 개발 음식 활용 등의 혜택도 주어진다.",개발 음식 활용 혜택만 주어진다.,contradiction
159517,159517,"힐링푸드 전문가 양성 교육 참여, 개발 음식 활용 등의 혜택도 주어진다.",힐링푸드 전문가에게 혜택이 주어진다.,neutral
159518,159518,"힐링푸드 전문가 양성 교육 참여, 개발 음식 활용 등의 혜택도 주어진다.",혜택에는 힐링푸드 전문가 양성 교육 참여가 있다.,entailment
159519,159519,힛걸 진심 최고다 그 어떤 히어로보다 멋지다,힛걸 그 어떤 히어로보다 별로다.,contradiction


In [24]:
test

,index,premise,hypothesis,label
0,0,다만 조금 좁아서 케리어를 펼치기 불편합니다.,케리어를 펼치기에 공간이 충분했습니다.,answer
1,1,그리고 위치가 시먼역보다는 샤오난먼역에 가까워요,시먼역보다는 샤오난먼역에 먼저 도착할 수 있어요,answer
2,2,구구절절 설명하고 이해시키려는 노력이 큰 의미없이 다가온다.,무엇인가 말을 많이 하기는 했지만 큰 의미가 있지는 않았다.,answer
3,3,몇 번을 다시봐도 볼 때마다 가슴이 저민다.,다시 봤을때는 무덤덤했다.,answer
4,4,"8월 중에 입주신청을 하면 청년은 9월, 신혼부부는 10월부터 입주가 가능하다.",8월 중에 입주신청을 하면 신혼부부는 9월 부터 입주가 가능하다.,answer
...,...,...,...,...
1661,1661,"또 작업자의 숙련도와 경험 향상, 전문성을 요구하는 난이도 높은 데이터 가공을 통해...",이미지 데이터를 가공하는 것이 가장 난이도가 높다.,answer
1662,1662,결말을 보니 아무래도 이 영화는 2부가 계획된 듯 합니다.,결말을 보니 분명히 2편이 나올것이 틀림없다.,answer
1663,1663,사회적 거리 두기 상황에서 총리도 카페를 갔다가 자리가 없어서 퇴짜 맞은 일도 있을...,사회적 거리두기 상황에서는 고위관직자도 방역지침을 준수해야 한다.,answer
1664,1664,로마에서 3박4일간 이곳에서 머물렀습니다.,이곳에서 머무르며 로마의 명소들을 방문했습니다.,answer


In [28]:
trans_list=[]
kor_to_trans(test['premise'], 'en',0,len(test))
np.save(data_path+'kor_to_eng_test_premise.npy',trans_list)

trans_list=[]
kor_to_trans(test['hypothesis'], 'en',0,len(test))
np.save(data_path+'kor_to_eng_test_hypothesis.npy',trans_list)

  0%|          | 0/1666 [00:00<?, ?it/s]

  0%|          | 0/1666 [00:00<?, ?it/s]

In [37]:
eng_premise =np.load(data_path+'kor_to_eng_test_premise.npy')
eng_hypothesis =np.load(data_path+'kor_to_eng_test_hypothesis.npy')


eng_data1=pd.DataFrame(eng_premise,columns = ['premise'])
eng_data2=pd.DataFrame(eng_hypothesis,columns = ['hypothesis'])
eng_data = pd.concat([eng_data1,eng_data2],axis=1)

In [39]:
eng_data

,premise,hypothesis
0,"However, it is a little narrow, so it is incon...",There was enough space to unfold the care.
1,And the location is closer to Xiaonanman stati...,You can arrive at Xiaonanmen Station before Si...
2,Efforts to explain and understand every phrase...,"I talked a lot about something, but it didn't ..."
3,"No matter how many times I watch it again, my ...","When I saw it again, I was calm."
4,"If you apply for occupancy in August, young pe...","If you apply for occupancy in August, newlywed..."
...,...,...
1661,"In addition, you can earn enough income throug...",Processing image data is the most difficult.
1662,"Looking at the ending, I think the second part...","Looking at the ending, there must be two films."
1663,"In a social distancing situation, it is said t...","In the situation of social distancing, high-ra..."
1664,I stayed here for 3 nights and 4 days in Rome.,I stayed here and visited Roman attractions.


In [41]:
eng_data.to_csv(data_path+'test_trans.csv')

In [44]:
eng_data[eng_data['hypothesis']==""].count()

premise       27
hypothesis    27
dtype: int64